In [ ]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen


import tensorflow as tf
import tensorflow_hub as hub



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#go to desired folder
%cd drive/MyDrive/tf

/content/drive/MyDrive/tf


In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

In [ ]:
#run model builder test
%cd ../..
!python models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
#paths
pipeline='inference_graphs/inference_graph_ssd_mobilenet_v2/pipeline.config'
checkpoint='inference_graphs/inference_graph_ssd_mobilenet_v2/checkpoint'
output='intermediate_models/ssd_mobilenet_v2'

In [ ]:
#the script below turns a TF model to a TFLite friendly format, needed for the conversion

In [ ]:
!python models/research/object_detection/export_tflite_graph_tf2.py \
    --pipeline_config_path {pipeline} \
    --trained_checkpoint_dir {checkpoint} \
    --output_directory {output}

In [ ]:
#path to load from
path='intermediate_models/ssd_mobilenet_v2/saved_model'
# where to put the tflite file
tflite_file_path="tflite_models/ssd_mobilenet_v2.tflite"

# get the converter with the model path
converter = tf.lite.TFLiteConverter.from_saved_model(path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

#convert the model
tflite_model = converter.convert()

# Save the model.
open(tflite_file_path, "wb").write(tflite_model)

In [ ]:
#if more information about the converter is needed
print(help(tf.lite.TFLiteConverter))